In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '../../python-tools'))
	print(os.getcwd())
except:
	pass


 ### 1. Changing string representation
 * To change the string representation of an instance, define the **__str__()** and ** __repr__()** methods.

In [1]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    # The format code {0.x} specifies the x-attribute of argument 0.
    # So, in the following function, the 0 is actually the instance self
    def __repr__(self):
        return 'Pair({0.x!r}, {0.y!r})'.format(self)

    def __str__(self):
        return '({0.x!s}, {0.y!s})'.format(self)


In [2]:
p = Pair(3,4)
p   # __repr__() output


Pair(3, 4)

In [3]:
print(p)    # __str__() output


(3, 4)


In [4]:
# the special !r formatting code indicates that the output of __repr__()
# should be used instead of __str__(), the default
print('p is {0!r}'.format(p))
print('p is {0}'.format(p))


p is Pair(3, 4)
p is (3, 4)


In [5]:
# alternative way to this implementation, use the % operator as follows:
def __repr__(self):
    return 'Pair(%r, %r)' % (self.x, self.y)


 ### 2. Customizing string formatting
 * define the __format__() method on a class

In [6]:
_formats = {
    'ymd' : '{d.year}-{d.month}-{d.day}',
    'mdy' : '{d.month}/{d.day}/{d.year}',
    'dmy' : '{d.day}/{d.month}/{d.year}'
}


In [7]:
class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)



In [8]:
d = Date(2012, 12, 21)
format(d)


'2012-12-21'

In [9]:
format(d, 'mdy')


'12/21/2012'

In [10]:
'The date is {:ymd}'.format(d)


'The date is 2012-12-21'

In [11]:
'The date is {:mdy}'.format(d)


'The date is 12/21/2012'

In [12]:
from datetime import date


In [13]:
d = date(2019, 10, 1)
format(d)


'2019-10-01'

In [14]:
format(d,'%A, %B %d, %Y')


'Tuesday, October 01, 2019'

In [15]:
'The end is {:%d %b %Y}.'.format(d)


'The end is 01 Oct 2019.'

 ### 3. Saving memory when creating a large number of instances
 * adding the __slots__ attribute to the class definition

In [16]:
# Attribute names listed in the __slots__ specifier are internally mapped to
# specific indices within this array.
class Date:
    __slots__ = ['year', 'month', 'day']
    def __init__(self, year, month, day):
            self.year = year
            self.month = month
            self.day = day


 ### 4. Customizing access to an attribute

In [17]:
class Person:
    def __init__(self, first_name):
        self.first_name = first_name

    # getter function
    @property
    def first_name(self):
        return self._first_name

    # setter function
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value

    # delete function
    @first_name.deleter
    def first_name(self):
        raise AttributeError("Can't delete attribute")


In [18]:
a = Person('Mike')


In [19]:
a.first_name    # Calls the getter


'Mike'

In [20]:
a.first_name = 42   # Calls the setter


TypeError: Expected a string

In [21]:
del a.first_name


AttributeError: Can't delete attribute

 * Properties can also be defined for existing get and set methods.

In [22]:
class Person:
    def __init__(self, first_name):
            self.set_first_name(first_name)

    # Getter function
    def get_first_name(self):
        return self._first_name

    # Setter function
    def set_first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value

    # Deleter function (optional)
    def del_first_name(self):
        raise AttributeError("Can't delete attribute")

    # Make a property from existing get/set methods
    first_name = property(get_first_name, set_first_name, del_first_name)


In [23]:
# to inspect a class with a property, we can find the raw methods in the fget,
# fset, and fdel attributes of the property itself.
# Normally, we wouldn’t call fget or fset directly, but they are triggered
# automatically when the property is accessed.
Person.first_name.fget

<function __main__.Person.get_first_name(self)>

In [24]:
import math

In [25]:
# Properties can also be a way to define computed attributes. These are attributes
# that are not actually stored, but computed on demand.
class Circle:
    def __init__(self, radius):
        self.radius = radius
    @property
    def area(self):
        return math.pi * self.radius ** 2
    @property
    def perimeter(self):
        return 2 * math.pi * self.radius


In [26]:
c = Circle(4.0)
c.radius


4.0

In [27]:
c.area   #notice lack of ()


50.26548245743669

In [28]:
c.perimeter     #notice lack of ()


25.132741228718345

 ### 5. Calling method on parent class

In [29]:
class A:
    def spam(self):
        print('A.spam')

class B(A):
    def spam(self):
        print('B.spam')
        super().spam()      # Call parent spam()


In [30]:
# very common use of super() is in the handling of the __init__() method to make
# sure that parents are properly initialized
class A:
    def __init__(self):
        self.x = 0

class B(A):
    def __init__(self):
        super().__init__()
        self.y = 1

In [31]:
# another common use of super() is in code that overrides any of Python’s
# special methods
class Proxy:
    def __init__(self, obj):
        self._obj = obj

    # Delegate attribute lookup to internal obj
    def __getattr__(self, name):
        return getattr(self._obj, name)

    # Delegate attribute assignment
    def __setattr__(self, name, value):
        if name.startswith('_'):
            super().__setattr__(name, value)    #call original __setattr__
        else:
            setattr(self._obj, name, value)


 ### 6. Creating a new kind of class or instance attribute
 * create an entirely new kind of instance attribute, define its functionality in the form of a descriptor class.
 * Descriptors provide the underlying magic for most of Python’s class features, including __@classmethod__, __@staticmethod__, __@property__, and even the `__slots__` specification.

In [32]:
# descriptor attribute for an integer type-checked attribute
class Integer:
    def __init__(self, name):
        self.name = name

    def __get__(self, instance, name):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]

    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError('expected an int')
        instance.__dict__[self.name] = value

    def __delete__(self, instance):
        del instance.__dict__[self.name]


In [33]:
# To use a descriptor, instances of the descriptor are placed into a class
# definition as class variables.
class Point:
    x = Integer('x')
    y = Integer('y')
    def __init__(self, x, y):
        self.x = x
        self.y = y


In [34]:
p = Point(2,3)
p.x     # Calls Point.x.__get__(p,Point)


2

In [35]:
p.y = 5   # call Point.y__set__(p, 5)


In [36]:
p.x = 2.5   # call Point.x__set__(p, 2.5)



TypeError: expected an int

In [37]:
# descriptor for a type-checked attribute
class Typed:
    def __init__(self, name, expected_type):
        self.name = name
        self.expected_type = expected_type

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]

    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError('Expected '+str(self.expected_type))
        instance.__dict__[self.name] = value

    def delete(self, instance):
        del instance.__dict__[self.name]


In [38]:
# Class decorator that applies it to selected attributes
def typeassert(**kwargs):
    def decorate(cls):
        for name, expected_type in kwargs.items():
            # Attach a Typed descriptor to the class
            setattr(cls, name, Typed(name, expected_type))
        return cls
    return decorate



In [39]:
# Example use
@typeassert(name=str, shares=int, price=float)
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price


 ### 7. Using lazily computed properties

In [40]:
class lazyproperty:
    def __init__(self, func):
        self.func = func
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance, self.func.__name__, value)
            return value


In [41]:
# utilize this code
class Circle:
    def __init__(self, radius):
        self.radius = radius
    @lazyproperty
    def area(self):
        print('computing area:')
        return math.pi * self.radius ** 2

    @lazyproperty
    def perimeter(self):
        print('computing perimeter:')
        return 2 * math.pi * self.radius


In [42]:
c = Circle(4.0)
c.radius

4.0

In [43]:
c.area

computing area:


50.26548245743669

In [44]:
c.area


50.26548245743669

In [45]:
c.perimeter

computing perimeter:


25.132741228718345

In [46]:
# the messages “Computing area” and “Computing perimeter” only appear once
c.perimeter

25.132741228718345

In [47]:
vars(c)

{'radius': 4.0, 'area': 50.26548245743669, 'perimeter': 25.132741228718345}

In [48]:
del c.area
vars(c)

{'radius': 4.0, 'perimeter': 25.132741228718345}

In [49]:
## Delete the variable and see property trigger again
c.area


computing area:


50.26548245743669

 ### 8.Simplifying the initialization of data structures
 * generalize the initialization of data structures into a single `__init__()` function defined in a common base class.

In [50]:
class Structure:
# Class variable that specifies expected fields
    _fields = []
    def __init__(self, *args):
        if len(args) != len(self._fields):
            raise TypeError('Expected {} arguments'.format(len(self._fields)))
        # set the arguments
        for name, value in zip(self._fields, args):
            setattr(self, name, value)



In [51]:
class Stock(Structure):
    _fields = ['name', 'shares', 'price']

class Point(Structure):
    _fields = ['x','y']

class Circle(Structure):
    _fields = ['radius']
    def area(self):
        return math.pi * self.radius ** 2


In [52]:
s = Stock('SBUX', 50, 91.56)
p = Point(2,3)
c = Circle(4.5)


In [53]:
s2 = Stock('SBUX', 50)



TypeError: Expected 3 arguments

In [54]:
# map the keyword arguments so that they only correspond to the attribute names
# specified in _fields.
class Structure:
    _fields = []
    def __init__(self, *args, **kwargs):
        if len(args) > len(self._fields):
            raise TypeError('Expected {} arguments'.format(len(self._fields)))
        # Set all of the positional arguments
        for name, value in zip(self._fields, args):
            setattr(self, name, value)
        # Set the remaining keyword arguments
        for name in self._fields[len(args):]:
            setattr(self, name, kwargs.pop(name))
        # Check for any remaining unknown arguments
        if kwargs:
            raise TypeError('Invalid argument(s):{}'.format(','.join(kwargs)))


In [55]:
class Stock(Structure):
    _fields = ['name', 'shares', 'price']

s1 = Stock('SBUX', 50, 91.56)
s2 = Stock('SBUX', 50, price = 91.56)
s3 = Stock('SBUX', shares = 50, price = 91.56)

In [56]:
# use keyword arguments as a means for adding additional attributes to the
# structure not specified in _fields.
class Structure:
    # Class variable that specifies expected fields
    _fields = []
    def __init__(self, *args, **kwargs):
        if len(args) != len(self._fields):
            raise TypeError('Expected {} arguments'.format(len(self._fields)))
        # Set the arguments
        for name, value in zip(self._fields, args):
            setattr(self, name, value)
        # Set the additional arguments (if any)
        extra_args = kwargs.keys() - self._fields
        for name in extra_args:
            setattr(self, name, kwargs.pop(name))
        if kwargs:
            raise TypeError('Duplicate values for {}'.format(','.join(kwargs)))

In [57]:
class Stock(Structure):
    _fields = ['name', 'shares', 'price']

s1 = Stock('SBUX', 50, 91.56)
s2 = Stock('SBUX', 50, 91.56, date='9/19/2019')


In [58]:
# instead of using the setattr() function, access the instance dictionary directly
class Structure:
    # Class variable that specifies expected fields
    _fields= []
    def __init__(self, *args):
        if len(args) != len(self._fields):
            raise TypeError('Expected {} arguments'.format(len(self._fields)))
        # Set the arguments (alternate)
        self.__dict__.update(zip(self._fields,args))


 ### 9. Calling a method on an object
 * use __getattr()__ for simple cases

In [59]:
class Point:
    def __init__(self, x,y):
        self.x = x
        self.y = y

    def __repr__(self):
        return 'Point({!r:},{!r:})'.format(self.x, self.y)

    def distance(self, x, y):
        return math.hypot(self.x - x, self.y - y)

In [60]:
p = Point(2,3)
d = getattr(p,'distance')(0,0)      #calling p.distance(0,0)


In [61]:
# alternative approach: use operator.methodcaller()
import operator
operator.methodcaller('distance', 0, 0)(p)


3.605551275463989

In [62]:
points = [
    Point(1,2),
    Point(3,0),
    Point(10,-3),
    Point(-5,-7),
    Point(-1,8),
    Point(3,2)
]
# sort by distance from origin (0,0)
points.sort(key=operator.methodcaller('distance', 0, 0))
points


[Point(1,2), Point(3,0), Point(3,2), Point(-1,8), Point(-5,-7), Point(10,-3)]

 ### 10. Implementing custom containers
 * notable classes defined in __collections__ include:
 ```
   * Sequence
   * MutableSequence
   * Mapping
   * MutableMapping
   * Set
   * MutableSet
 ```

In [63]:
import collections
import bisect


In [64]:
# simply instantiate any of these classes to see what methods need to be implemented
# to make a custom container with that behavior

class SortedItems(collections.Sequence):
    def __init__(self, initial = None):
        self._items = sorted(initial) if initial else []

    # required sequence methods
    def __getitem__(self, index):
        return self._items[index]

    def __len__(self):
        return len(self._items)

    # Method for adding an item in the right location
    def add(self, item):
        bisect.insort(self._items,item)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  after removing the cwd from sys.path.


In [65]:
items = SortedItems([14,1,9])
list(items)


[1, 9, 14]

In [66]:
print(items[0])
print(items[-1])

1
14


In [67]:
items.add(10)
list(items)

[1, 9, 10, 14]

In [68]:
items.add(-3)
list(items)

[-3, 1, 9, 10, 14]

In [69]:
items[1:4]

[1, 9, 10]

In [70]:
3 in items

False

In [71]:
for n in items:
    print(n)

-3
1
9
10
14


In [72]:
# many of the abstract base classes in collections also provide default
# implementations of common container methods.
# for example, a class that inherits from collections.MutableSequence
class Items(collections.MutableSequence):
    def __init__(self, initial = None):
        self._items = list(initial) if initial is not None else []

    # Required sequence methods
    def __getitem__(self, index):
        print('Getting:', index)
        return self._items[index]

    def __setitem__(self, index, value):
        print('Setting:', index, value)
        self._items[index] = value

    def __delitem__(self, index):
        print('Deleting:', index)
        del self._items[index]

    def insert(self, index, value):
        print('Inserting:',index, value)
        self._items.insert(index, value)

    def __len__(self):
        print('len')
        return len(self._items)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  after removing the cwd from sys.path.


In [73]:
# create an instance of Items, it supports almost all of the core list methods
# (e.g., append(), remove(), count(), etc.). These methods are implemented in
# such a way that they only use the required ones.
a = Items([1,2,3])
len(a)

len


3

In [74]:
a.append(4)

len
Inserting: 3 4


In [75]:
a.count(2)

Getting: 0
Getting: 1
Getting: 2
Getting: 3
Getting: 4


1

In [76]:
a.remove(3)

Getting: 0
Getting: 1
Getting: 2
Deleting: 2


 ### 11. Defining more than one constructor in a class

In [77]:
import time

class Date:
    # primary constructor
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    # alternative constructor
    @classmethod
    def today(cls):
        t = time.localtime()
        return cls(t.tm_year, t.tm_mon, t.tm_mday)


In [78]:
d = Date(2019,10,1)
t = Date.today()
print(t.year, t.month, t.day)

2019 9 25


 ### 12. Creating an instance without invoking init
 * calling the `__new__()` method

In [79]:
class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day


In [80]:
d = Date.__new__(Date)
d


In [81]:
# set appropriate instance variables
data = {'year':2019, 'month':9, 'day':20}
for key, value in data.items():
    setattr(d, key, value)
d.year


2019

In [82]:
# very useful when instances are being created in a nonstandard way, such as
# when deserializing data or inmplementation of a class method that's been defined
# as an alternate constructor, as bypassing __init__() might causes problems arise
class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    @classmethod
    def today(cls):
        d = cls.__new__(cls)
        t = time.localtime()
        d.year = t.tm_year
        d.month = t.tm_mon
        d.day = t.tm_mday
        return d

 ### 13. Implementing a data model or type system
 * to customize the setting of attributes on a per-attribute basis

In [83]:
# base class. uses a descriptor to set a value
class Descriptor:
    def __init__(self, name=None, **opts):
        self.name = name
        for key, value in opts.items():
            setattr(self, key, value)
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value


In [84]:
# descriptor for enforcing types
class Typed(Descriptor):
    expected_type = type(None)

    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError('expected '+str(self.expected_type))
        super().__set__(instance, value)


In [85]:
# descriptor for enforcing values
class Unsigned(Descriptor):
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Expected >= 0')
        super().__set__(instance, value)

class MaxSized(Descriptor):
    def __init__(self, name=None, **opts):
        if 'size' not in opts:
            raise TypeError('missing size option')
        super().__init__(name, **opts)

    def __set__(self, instance, value):
        if len(value) >= self.size:
            raise ValueError('size must be < ' +str(self.size))
        super().__set__(instance, value)

In [86]:
# some implements
class Integer(Typed):
    expected_type = int

class UnsignedInteger(Integer, Unsigned):
    pass

class Float(Typed):
    expected_type = float

class UnsignedFloat(Float, Unsigned):
    pass

class String(Typed):
    expected_type = str

class SizedString(String, MaxSized):
    pass



In [87]:
# use these type objects to define a class
class Stock:
    # specify constraints
    name = SizedString('name', size = 8)
    shares = UnsignedInteger('shares')
    price = UnsignedFloat('price')
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

In [88]:
s = Stock('SBUX', 50, 91.56)
s.name


'SBUX'

In [89]:
s.name = 'ABCDABCDE'


ValueError: size must be < 8

In [90]:
# Class decorator to apply constraints
def check_attributes(**kwargs):
    def decorate(cls):
        for key, value in kwargs.items():
            if isinstance(value, Descriptor):
                value.name = key
                setattr(cls, key, value)
            else:
                setattr(cls, key, value(key))
        return cls
    return decorate


In [91]:
# example

@check_attributes(name = SizedString(size=8),
                  shares = UnsignedInteger,
                  price = UnsignedFloat)
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares,
        self.price = price



In [93]:
# Another approach to simplify the specification of constraints is to use a metaclass

# A metaclass that applies checking
class checkedmeta(type):
    def __new__(cls, clsname, bases, methods):
        # attach attribute names to the descriptors
        for key, value in methods.items():
            if isinstance(value, Descriptor):
                value.name = key
            return type.__new__(cls, clsname, bases, methods)

In [94]:
# example
class Stock(metaclass = checkedmeta):
    name = SizedString(size=8)
    shares = UnsignedInteger()
    price = UnsignedFloat()
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price
